<a href="https://colab.research.google.com/github/Naveen142-12/Mental-health-campanion/blob/main/Mental_health_companion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install groq gradio gtts speechrecognition pydub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 53.7 MB/s eta 0:00:00


In [ ]:
import tempfile
import gradio as gr
from gtts import gTTS
import groq
import speech_recognition as sr
from pydub import AudioSegment
import time
import os

# Init Groq client
client = groq.Groq(api_key="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")  # Replace with your Groq key

# Initialize speech recognizer
recognizer = sr.Recognizer()

def speech_to_text(audio_path):
    try:
        # Check if audio file exists and has content
        if not audio_path or not os.path.exists(audio_path) or os.path.getsize(audio_path) == 0:
            return ""

        # Convert to WAV format
        audio = AudioSegment.from_file(audio_path)
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as wav_file:
            temp_wav = wav_file.name
            audio.export(temp_wav, format="wav")

            # Verify the converted file
            if os.path.getsize(temp_wav) == 0:
                return ""

            with sr.AudioFile(temp_wav) as source:
                # Adjust for ambient noise and record
                recognizer.adjust_for_ambient_noise(source)
                audio_data = recognizer.record(source)
                text = recognizer.recognize_google(audio_data)
                return text
    except Exception as e:
        print(f"Speech recognition error: {str(e)}")
        return ""

def chatbot_response(audio_input, text_input, history, voice_enabled):
    time.sleep(0.3)  # Small delay for stability

    # Get message from either input
    message = text_input if text_input else speech_to_text(audio_input)
    if not message.strip():
        return None, "", history, history, None

    # Get AI response
    response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {"role": "system", "content": "You are a cheerful, caring best friend. Talk like a human buddy. Respond warmly to emotional statements like 'I feel anxious', giving comforting words and easy ideas."},
            {"role": "user", "content": message}
        ]
    )

    reply = response.choices[0].message.content.strip()
    history.append((message, reply))

    # Generate voice response if enabled
    audio_path = None
    if voice_enabled:
        try:
            tts = gTTS(text=reply, lang="en", slow=False)
            with tempfile.NamedTemporaryFile(suffix=".mp3", delete=False) as fp:
                audio_path = fp.name
                tts.save(audio_path)
                time.sleep(0.2)  # Ensure file is fully written
        except Exception as e:
            print(f"Voice generation error: {e}")

    return None, "", history, history, audio_path

# Full-width UI
css = """
.gradio-container {
    max-width: 100% !important;
    padding: 20px !important;
}
#chatbot {
    min-height: 400px;
    width: 100% !important;
}
.control-row {
    margin-top: 15px;
    gap: 10px;
}
.tab-content {
    padding: 15px 0;
}
"""

with gr.Blocks(theme=gr.themes.Soft(), css=css) as demo:
    gr.Markdown("""
    <div style="text-align: center;">
        <h1>🤖 Mental Health Companion</h1>
        <p><i>Your friendly AI buddy here to listen and support you</i></p>
    </div>
    """)

    # Chat interface
    chatbot = gr.Chatbot(elem_id="chatbot", height=400)

    # Input tabs
    with gr.Tabs():
        with gr.Tab("🎤 Voice Input"):
            audio_input = gr.Audio(
                sources=["microphone"],
                type="filepath",
                label="Speak your feelings (click to record)",
                interactive=True,
                show_download_button=False
            )
        with gr.Tab("✏️ Text Input"):
            text_input = gr.Textbox(
                placeholder="How are you feeling today?",
                label="Your Message",
                lines=3,
                container=False,
                autofocus=True
            )

    # Controls
    with gr.Row():
        submit_btn = gr.Button("Send", variant="primary", scale=1)
        clear_btn = gr.Button("Clear Chat", scale=1)
        voice_enabled = gr.Checkbox(
            label="Enable Voice Responses",
            value=True,
            scale=0,
            interactive=True
        )

    audio_output = gr.Audio(
        label="AI Response",
        autoplay=True,
        visible=True,
        interactive=False
    )
    state = gr.State([])

    # Event handlers
    submit_btn.click(
        chatbot_response,
        [audio_input, text_input, state, voice_enabled],
        [audio_input, text_input, chatbot, state, audio_output]
    )

    clear_btn.click(
        lambda: (None, "", [], [], None),
        None,
        [audio_input, text_input, chatbot, state, audio_output]
    )

demo.launch(debug=True)

<ipython-input-3-bcd9e354a5d1>:104: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(elem_id="chatbot", height=400)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://225b49d65cb7544297.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
